<a href="https://colab.research.google.com/github/oluwoleowoeye/food-price-analysis/blob/main/Food_price_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()



Saving Project_food prices anomalies indicator.xlsx to Project_food prices anomalies indicator.xlsx


In [2]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('Project_food prices anomalies indicator.xlsx')

# Check available sheet names
print(xls.sheet_names)



['CodeDescriptions', 'Goal2']


In [4]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile('Project_food prices anomalies indicator.xlsx')

# Load the sheets using their exact names
df_main = pd.read_excel(xls, sheet_name='Goal2')          # Main dataset
df_codes = pd.read_excel(xls, sheet_name='CodeDescriptions')  # Code descriptions

# Verify the data
print("Main Data (Goal2):")
print(df_main.head())

print("\nCode Descriptions:")
print(df_codes.head())


Main Data (Goal2):
   Goal Target Indicator   SeriesCode  \
0     2    2.c     2.c.1  AG_FPA_CFPI   
1     2    2.c     2.c.1  AG_FPA_CFPI   
2     2    2.c     2.c.1  AG_FPA_CFPI   
3     2    2.c     2.c.1  AG_FPA_CFPI   
4     2    2.c     2.c.1  AG_FPA_CFPI   

                                   SeriesDescription  GeoAreaCode  \
0  Indicator of Food Price Anomalies (IFPA), by C...            4   
1  Indicator of Food Price Anomalies (IFPA), by C...            4   
2  Indicator of Food Price Anomalies (IFPA), by C...            4   
3  Indicator of Food Price Anomalies (IFPA), by C...            4   
4  Indicator of Food Price Anomalies (IFPA), by C...            4   

   GeoAreaName  TimePeriod  Value  Time_Detail  ...  BasePeriod  \
0  Afghanistan        2010  -0.12         2010  ...         NaN   
1  Afghanistan        2011  -0.17         2011  ...         NaN   
2  Afghanistan        2012  -0.28         2012  ...         NaN   
3  Afghanistan        2013   0.30         2013  ...

In [7]:
# Check missing values
print("Missing values in Main Data (Goal2):")
print(df_main.isnull().sum())

# --- Handle missing values ---
df_main_cleaned = df_main.copy()

# A. Fill numerical columns (e.g., 'Value', 'UpperBound', 'LowerBound')
numerical_cols = ['Value', 'UpperBound', 'LowerBound']
for col in numerical_cols:
    df_main_cleaned[col] = df_main_cleaned[col].fillna(df_main_cleaned[col].mean())

# B. Fill categorical columns (e.g., 'Observation Status', 'Reporting Type')
categorical_cols = ['Observation Status', 'Reporting Type', 'Units']
for col in categorical_cols:
    df_main_cleaned[col] = df_main_cleaned[col].fillna("Unknown")

# C. Drop rows if critical columns are missing (e.g., 'SeriesCode', 'TimePeriod')
critical_cols = ['SeriesCode', 'TimePeriod', 'GeoAreaCode']
df_main_cleaned = df_main_cleaned.dropna(subset=critical_cols)

# Verify
print("\nMissing values after cleaning:")
print(df_main_cleaned.isnull().sum())

Missing values in Main Data (Goal2):
Goal                           0
Target                         0
Indicator                      0
SeriesCode                     0
SeriesDescription              0
GeoAreaCode                    0
GeoAreaName                    0
TimePeriod                     0
Value                        285
Time_Detail                    0
TimeCoverage                5817
UpperBound                  5817
LowerBound                  5817
BasePeriod                  5817
Source                       285
GeoInfoUrl                  5817
FootNote                       0
Nature                       285
Observation Status             0
Reporting Type                 0
Severity of price levels    5154
Type of product             3477
Units                          0
dtype: int64

Missing values after cleaning:
Goal                           0
Target                         0
Indicator                      0
SeriesCode                     0
SeriesDescription          

In [9]:
# Merge using the correct key columns:
merged_df = pd.merge(
    df_main_cleaned,
    df_codes,
    left_on='SeriesCode',   # Column in df_main_cleaned
    right_on='Code',        # Column in df_codes (previously called 'CodeColumn')
    how='left'
)

# Verify the merge
print("Merged Data Columns:", merged_df.columns.tolist())
print("\nSample Rows:")
print(merged_df.head())

Merged Data Columns: ['Goal', 'Target', 'Indicator', 'SeriesCode', 'SeriesDescription', 'GeoAreaCode', 'GeoAreaName', 'TimePeriod', 'Value', 'Time_Detail', 'TimeCoverage', 'UpperBound', 'LowerBound', 'BasePeriod', 'Source', 'GeoInfoUrl', 'FootNote', 'Nature', 'Observation Status', 'Reporting Type', 'Severity of price levels', 'Type of product', 'Units', 'Dimension', 'Code', 'Description']

Sample Rows:
   Goal Target Indicator   SeriesCode  \
0     2    2.c     2.c.1  AG_FPA_CFPI   
1     2    2.c     2.c.1  AG_FPA_CFPI   
2     2    2.c     2.c.1  AG_FPA_CFPI   
3     2    2.c     2.c.1  AG_FPA_CFPI   
4     2    2.c     2.c.1  AG_FPA_CFPI   

                                   SeriesDescription  GeoAreaCode  \
0  Indicator of Food Price Anomalies (IFPA), by C...            4   
1  Indicator of Food Price Anomalies (IFPA), by C...            4   
2  Indicator of Food Price Anomalies (IFPA), by C...            4   
3  Indicator of Food Price Anomalies (IFPA), by C...            4   
4 

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Normalize 'Value' to [0, 1]
scaler = MinMaxScaler()
merged_df['Value_Normalized'] = scaler.fit_transform(merged_df[['Value']])

# Optional: Standardize 'UpperBound' (mean=0, std=1)
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# merged_df['UpperBound_Standardized'] = scaler.fit_transform(merged_df[['UpperBound']])

print("\nNormalized Data Sample:")
print(merged_df[['Value', 'Value_Normalized']].head())


Normalized Data Sample:
   Value  Value_Normalized
0  -0.12          0.357010
1  -0.17          0.356689
2  -0.28          0.355982
3   0.30          0.359707
4   0.42          0.360478
